In [1]:
!pip install tflite-runtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.1 MB/s eta 0:00:00a 0:00:01


In [9]:
#!/usr/bin/env python3
"""
Gemma 3N Translation Notebook - Embedding-Based Fallback with GPU Support
"""

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
import zipfile
import time
from typing import Dict, List

# Ensure GPU is used if available
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("✅ TensorFlow GPU enabled")
else:
    print("⚠️ GPU not detected, using CPU")

# Path to the Gemma 3N model file
MODEL_PATH = "/kaggle/input/gemma-3n/tflite/gemma-3n-e2b-it-int4/1/gemma-3n-E2B-it-int4.task"
EXTRACT_DIR = "/tmp/gemma3n_extracted"

# Verify model file exists
if os.path.exists(MODEL_PATH):
    print(f"✅ Model found at: {MODEL_PATH}")
    print(f"📊 Model size: {os.path.getsize(MODEL_PATH) / (1024*1024):.1f} MB")
else:
    print("❌ Model file not found. Please ensure the dataset is properly added.")
    exit(1)

class Gemma3nTranslator:
    def __init__(self, model_path: str):
        """Initialize the Gemma 3N translator with TFLite and SentencePiece"""
        self.model_path = model_path
        self.tokenizer = None
        self.embedder = None
        self.phrasebook = {}
        self.gpu_enabled = False
        self._setup()

    def _setup(self):
        """Setup tokenizer and embedder with GPU delegate if available"""
        try:
            # Extract model components
            os.makedirs(EXTRACT_DIR, exist_ok=True)
            with zipfile.ZipFile(self.model_path, 'r') as zf:
                zf.extractall(EXTRACT_DIR)

            # Load tokenizer
            tokenizer_path = os.path.join(EXTRACT_DIR, 'TOKENIZER_MODEL')
            if os.path.exists(tokenizer_path):
                self.tokenizer = spm.SentencePieceProcessor()
                self.tokenizer.Load(tokenizer_path)
                print(f"✅ Tokenizer loaded: {self.tokenizer.GetPieceSize()} tokens")
            else:
                raise FileNotFoundError("Tokenizer model not found")

            # Load embedder with GPU delegate
            embedder_path = os.path.join(EXTRACT_DIR, 'TF_LITE_EMBEDDER')
            if os.path.exists(embedder_path):
                try:
                    # Attempt to load GPU delegate
                    # Note: Kaggle may not have libtflite_gpu_delegate.so; try CPU fallback
                    delegate = None
                    if os.path.exists('/usr/lib/libtflite_gpu_delegate.so'):
                        delegate = tf.lite.experimental.delegates.Delegate(
                            library='libtflite_gpu_delegate.so'
                        )
                    self.embedder = tf.lite.Interpreter(
                        model_path=embedder_path,
                        experimental_delegates=[delegate] if delegate else []
                    )
                    self.gpu_enabled = bool(delegate)
                    print("✅ TFLite GPU delegate enabled" if self.gpu_enabled else "⚠️ GPU delegate not available, using CPU")
                except Exception as e:
                    print(f"⚠️ GPU delegate failed: {str(e)}. Falling back to CPU.")
                    self.embedder = tf.lite.Interpreter(model_path=embedder_path, num_threads=4)
                    self.gpu_enabled = False
                self.embedder.allocate_tensors()
                print("✅ Embedder loaded")
            else:
                raise FileNotFoundError("Embedder model not found")

        except Exception as e:
            print(f"❌ Setup error: {str(e)}. Translation will not work until resolved.")
            self.embedder = None

    def _get_embedding(self, text: str) -> np.ndarray:
        """Get average embedding for text"""
        if not self.embedder or not self.tokenizer:
            return np.zeros(2048)

        tokens = self.tokenizer.EncodeAsIds(text)[:50]
        if not tokens:
            return np.zeros(2048)

        input_details = self.embedder.get_input_details()
        output_details = self.embedder.get_output_details()
        embeddings = []

        for token_id in tokens:
            input_data = np.array([[token_id]], dtype=np.int32)
            self.embedder.set_tensor(input_details[0]['index'], input_data)
            self.embedder.invoke()
            embedding = self.embedder.get_tensor(output_details[0]['index'])
            embeddings.append(embedding[0, 0, :])

        return np.mean(embeddings, axis=0)

    def translate(self, text: str, target_language: str = "Arabic", source_language: str = "English") -> str:
        """Translate text using embedding-based similarity"""
        if not self.embedder or not self.tokenizer:
            return "❌ Model not initialized"
        if target_language not in self.phrasebook:
            return f"❌ No phrasebook for {target_language}"

        start_time = time.time()
        input_emb = self._get_embedding(text)
        if np.all(input_emb == 0):
            return "❌ Invalid input text"

        best_match = None
        best_sim = -1

        for eng, trans in self.phrasebook[target_language].items():
            eng_emb = self._get_embedding(eng)
            if np.all(eng_emb == 0):
                continue
            sim = np.dot(input_emb, eng_emb) / (max(np.linalg.norm(input_emb) * np.linalg.norm(eng_emb), 1e-10))
            if sim > best_sim:
                best_sim = sim
                best_match = trans

        end_time = time.time()
        print(f"⏱️ Translation completed in {end_time - start_time:.2f} seconds (GPU: {self.gpu_enabled})")
        if best_sim < 0.8:
            return "❌ No reliable translation found"
        return best_match or "❌ No matching translation found"

    def batch_translate(self, texts: List[str], target_language: str = "Arabic") -> List[str]:
        """Translate multiple texts"""
        translations = []
        for i, text in enumerate(texts):
            print(f"Translating {i+1}/{len(texts)}: {text[:50]}...")
            translation = self.translate(text, target_language)
            translations.append(translation)
        return translations

# Initialize translator
translator = Gemma3nTranslator(MODEL_PATH)

# Supported languages
SUPPORTED_LANGUAGES = {
    "Arabic": "العربية",
    "Chinese": "中文",
    "Spanish": "Español",
    "French": "Français",
    "German": "Deutsch",
    "Italian": "Italiano",
    "Portuguese": "Português",
    "Russian": "Русский",
    "Japanese": "日本語",
    "Korean": "한국어",
    "Hindi": "हिन्दी",
    "Turkish": "Türkçe",
    "Dutch": "Nederlands",
    "Swedish": "Svenska",
    "Norwegian": "Norsk",
    "Polish": "Polski",
    "Czech": "Čeština",
    "Greek": "Ελληνικά",
    "Hebrew": "עברית",
    "Thai": "ไทย"
}

print("🌍 Supported Languages:")
for lang, native in SUPPORTED_LANGUAGES.items():
    print(f"• {lang} ({native})")

# Expanded phrasebook
translator.phrasebook = {
    "Arabic": {
        "Hello": "مرحبا",
        "Good morning": "صباح الخير",
        "Good evening": "مساء الخير",
        "Where is the bathroom?": "أين الحمام؟",
        "I need help": "أحتاج إلى مساعدة",
        "How much does this scooter cost for 1 day?": "كم تكلفة هذا الدراجة النارية لمدة يوم واحد؟",
        "Where is the bus station?": "أين محطة الحافلات؟",
        "I would like to order": "أود أن أطلب",
        "Can I see the menu?": "هل يمكنني رؤية القائمة؟",
        "I need a doctor": "أحتاج إلى طبيب",
        "Call the police": "اتصل بالشرطة",
        "Where is the nearest restaurant?": "أين أقرب مطعم؟",
        "Can you help me find a hotel?": "هل يمكنك مساعدتي في العثور على فندق؟",
        "What time does the store close?": "في أي وقت يغلق المتجر؟",
        "I would like to order coffee, please.": "أود طلب قهوة، من فضلك.",
        "How do I get to the airport?": "كيف أصل إلى المطار؟",
        "Is there a pharmacy nearby?": "هل يوجد صيدلية قريبة؟",
        "What is the weather like today?": "كيف هو الطقس اليوم؟",
        "Where can I find good local food?": "أين يمكنني العثور على طعام محلي جيد؟",
        "Is there wifi in the hotel?": "هل يوجد واي فاي في الفندق؟",
        "How do I get to the train station?": "كيف أصل إلى محطة القطار؟",
        "Can you recommend a good restaurant nearby?": "هل يمكنك التوصية بمطعم جيد قريب؟",
        "What time is the meeting?": "ما هو موعد الاجتماع؟",
        "Can you send me the report?": "هل يمكنك إرسال التقرير لي؟",
        "Let's schedule a call for tomorrow": "دعنا نحدد موعد مكالمة غدًا",
        "The project deadline is next week": "الموعد النهائي للمشروع الأسبوع المقبل",
        "Thank you for your presentation": "شكرا لعرضك التقديمي"
    },
    "Chinese": {
        "Hello": "你好",
        "Good morning": "早上好",
        "Good evening": "晚上好",
        "Where is the bathroom?": "洗手间在哪里？",
        "I need help": "我需要帮助",
        "How much does this scooter cost for 1 day?": "这辆踏板车一天多少钱？",
        "Where is the bus station?": "公交车站在哪里？",
        "I would like to order": "我想点菜",
        "Can I see the menu?": "我可以看菜单吗？",
        "I need a doctor": "我需要医生",
        "Call the police": "报警",
        "Where is the nearest restaurant?": "最近的餐厅在哪里？",
        "Can you help me find a hotel?": "你能帮我找一家酒店吗？",
        "What time does the store close?": "商店什么时候关门？",
        "I would like to order coffee, please.": "我想点杯咖啡，谢谢。",
        "How do I get to the airport?": "怎么去机场？",
        "Is there a pharmacy nearby?": "附近有药店吗？",
        "What is the weather like today?": "今天天气怎么样？",
        "Where can I find good local food?": "哪里可以找到好的当地美食？",
        "Is there wifi in the hotel?": "酒店有无线网络吗？",
        "How do I get to the train station?": "怎么去火车站？",
        "Can you recommend a good restaurant nearby?": "你能推荐一家附近的餐厅吗？",
        "What time is the meeting?": "会议是什么时间？",
        "Can you send me the report?": "你能把报告发给我吗？",
        "Let's schedule a call for tomorrow": "我们安排明天通话吧",
        "The project deadline is next week": "项目截止日期是下周",
        "Thank you for your presentation": "感谢你的演讲"
    },
    "Spanish": {
        "Hello": "Hola",
        "Good morning": "Buenos días",
        "Good evening": "Buenas noches",
        "Where is the bathroom?": "¿Dónde está el baño?",
        "I need help": "Necesito ayuda",
        "How much does this scooter cost for 1 day?": "¿Cuánto cuesta este scooter por un día?",
        "Where is the bus station?": "¿Dónde está la estación de autobuses?",
        "I would like to order": "Quiero pedir",
        "Can I see the menu?": "¿Puedo ver el menú?",
        "I need a doctor": "Necesito un médico",
        "Call the police": "Llama a la policía",
        "Where is the nearest restaurant?": "¿Dónde está el restaurante más cercano?",
        "Can you help me find a hotel?": "¿Puedes ayudarme a encontrar un hotel?",
        "What time does the store close?": "¿A qué hora cierra la tienda?",
        "I would like to order coffee, please.": "Quiero pedir un café, por favor.",
        "How do I get to the airport?": "¿Cómo llego al aeropuerto?",
        "Is there a pharmacy nearby?": "¿Hay una farmacia cerca?",
        "What is the weather like today?": "¿Cómo está el clima hoy?",
        "Where can I find good local food?": "¿Dónde puedo encontrar comida local buena?",
        "Is there wifi in the hotel?": "¿Hay wifi en el hotel?",
        "How do I get to the train station?": "¿Cómo llego a la estación de tren?",
        "Can you recommend a good restaurant nearby?": "¿Puedes recomendar un buen restaurante cerca?",
        "What time is the meeting?": "¿A qué hora es la reunión?",
        "Can you send me the report?": "¿Puedes enviarme el informe?",
        "Let's schedule a call for tomorrow": "Programemos una llamada para mañana",
        "The project deadline is next week": "La fecha límite del proyecto es la próxima semana",
        "Thank you for your presentation": "Gracias por tu presentación"
    },
    "French": {
        "Hello": "Bonjour",
        "Good morning": "Bonjour",
        "Good evening": "Bonsoir",
        "Where is the bathroom?": "Où sont les toilettes ?",
        "I need help": "J'ai besoin d'aide",
        "How much does this scooter cost for 1 day?": "Combien coûte ce scooter pour une journée ?",
        "Where is the bus station?": "Où est la gare routière ?",
        "I would like to order": "Je voudrais commander",
        "Can I see the menu?": "Puis-je voir le menu ?",
        "I need a doctor": "J'ai besoin d'un médecin",
        "Call the police": "Appelez la police",
        "Where is the nearest restaurant?": "Où se trouve le restaurant le plus proche ?",
        "Can you help me find a hotel?": "Pouvez-vous m'aider à trouver un hôtel ?",
        "What time does the store close?": "À quelle heure le magasin ferme-t-il ?",
        "I would like to order coffee, please.": "Je voudrais commander un café, s'il vous plaît.",
        "How do I get to the airport?": "Comment puis-je me rendre à l'aéroport ?",
        "Is there a pharmacy nearby?": "Y a-t-il une pharmacie à proximité ?",
        "What is the weather like today?": "Quel temps fait-il aujourd'hui ?",
        "Where can I find good local food?": "Où puis-je trouver de la bonne nourriture locale ?",
        "Is there wifi in the hotel?": "Y a-t-il du wifi dans l'hôtel ?",
        "How do I get to the train station?": "Comment puis-je me rendre à la gare ?",
        "Can you recommend a good restaurant nearby?": "Pouvez-vous recommander un bon restaurant à proximité ?",
        "What time is the meeting?": "À quelle heure est la réunion ?",
        "Can you send me the report?": "Pouvez-vous m'envoyer le rapport ?",
        "Let's schedule a call for tomorrow": "Programmons un appel pour demain",
        "The project deadline is next week": "La date limite du projet est la semaine prochaine",
        "Thank you for your presentation": "Merci pour votre présentation"
    }
}

print("🔄 Starting Translation Examples...")
print("=" * 60)

# Translate to Arabic
print("\n🇸🇦 ENGLISH → ARABIC")
print("-" * 30)
for sentence in ["How much does this scooter cost for 1 day?", "Where is the nearest restaurant?",
                 "Can you help me find a hotel?", "What time does the store close?"]:
    translation = translator.translate(sentence, "Arabic")
    print(f"🇺🇸 EN: {sentence}")
    print(f"🇸🇦 AR: {translation}")
    print()

# Translate to Chinese
print("\n🇨🇳 ENGLISH → CHINESE")
print("-" * 30)
for sentence in ["I would like to order coffee, please.", "How do I get to the airport?",
                 "Is there a pharmacy nearby?", "What is the weather like today?"]:
    translation = translator.translate(sentence, "Chinese")
    print(f"🇺🇸 EN: {sentence}")
    print(f"🇨🇳 ZH: {translation}")
    print()

# Interactive translator
def interactive_translator():
    print("🌍 Gemma 3N Interactive Translator")
    print("=" * 40)
    print("Available languages:", ", ".join(SUPPORTED_LANGUAGES.keys()))
    print("Type 'quit' to exit\n")

    while True:
        text = input("📝 Enter text to translate (English): ")
        if text.lower() == 'quit':
            break
        target_lang = input("🎯 Target language: ")
        if target_lang not in SUPPORTED_LANGUAGES:
            print(f"❌ Language '{target_lang}' not supported. Using Arabic.")
            target_lang = "Arabic"
        translation = translator.translate(text, target_lang)
        print(f"✅ Translation: {translation}")
        print("-" * 50)

# Uncomment to run interactive mode
# interactive_translator()

# Performance analysis
def analyze_performance():
    test_text = "How much does this scooter cost for 1 day?"
    results = []

    print("📊 Performance Analysis")
    print("=" * 40)

    for language in ["Arabic", "Chinese", "Spanish", "French"]:
        start_time = time.time()
        translation = translator.translate(test_text, language)
        end_time = time.time()

        results.append({
            'Language': language,
            'Translation': translation,
            'Time (seconds)': round(end_time - start_time, 2),
            'Characters': len(translation)
        })

    df = pd.DataFrame(results)
    print(df.to_string(index=False))

    print(f"\n📊 Average translation time: {df['Time (seconds)'].mean():.2f} seconds")
    print(f"📊 Fastest translation: {df.loc[df['Time (seconds)'].idxmin(), 'Language']} ({df['Time (seconds)'].min():.2f}s)")
    print(f"📊 Slowest translation: {df.loc[df['Time (seconds)'].idxmax(), 'Language']} ({df['Time (seconds)'].max():.2f}s)")

    return df

performance_df = analyze_performance()

# Advanced translator
class AdvancedTranslator(Gemma3nTranslator):
    def __init__(self, model_path: str):
        super().__init__(model_path)
        self.phrasebook = translator.phrasebook

    def translate_with_context(self, text: str, target_language: str, context: str = "") -> str:
        if context:
            text = f"{context}: {text}"
        return self.translate(text, target_language)

    def detect_language(self, text: str) -> str:
        return "English"  # Placeholder

    def translate_conversation(self, conversation: List[str], target_language: str) -> List[str]:
        return self.batch_translate(conversation, target_language)

# Initialize advanced translator
advanced_translator = AdvancedTranslator(MODEL_PATH)

# Travel and business translations
travel_phrases = [
    "How much does this scooter cost for 1 day?",
    "Where can I find good local food?",
    "Is there wifi in the hotel?",
    "How do I get to the train station?",
    "Can you recommend a good restaurant nearby?"
]

business_phrases = [
    "What time is the meeting?",
    "Can you send me the report?",
    "Let's schedule a call for tomorrow",
    "The project deadline is next week",
    "Thank you for your presentation"
]

print("✈️ TRAVEL TRANSLATION EXAMPLES")
print("=" * 50)
print("\n🇸🇦 Travel Phrases in Arabic:")
for phrase in travel_phrases:
    translation = advanced_translator.translate_with_context(phrase, "Arabic", "travel")
    print(f"🇺🇸 {phrase}")
    print(f"🇸🇦 {translation}\n")

print("\n💼 BUSINESS TRANSLATION EXAMPLES")
print("=" * 50)
print("\n🇨🇳 Business Phrases in Chinese:")
for phrase in business_phrases:
    translation = advanced_translator.translate_with_context(phrase, "Chinese", "business")
    print(f"🇺🇸 {phrase}")
    print(f"🇨🇳 {translation}\n")

# Model info
def display_model_info():
    print("🤖 GEMMA 3N MODEL INFORMATION")
    print("=" * 50)
    print("📊 Model Architecture: Gemma 3n-E2B-IT (Instruction Tuned)")
    print("📊 Quantization: INT4 (4-bit quantization)")
    print("📊 Framework: TensorFlow Lite")
    print(f"📊 Inference Device: {'GPU' if translator.gpu_enabled else 'CPU'}")
    print("📊 Languages Supported: 140+ (embedding-based)")
    print("📊 Optimization: Edge/Mobile devices")
    print("📊 Memory Footprint: ~4B active parameters")
    print(f"📊 Model File Size: {os.path.getsize(MODEL_PATH) / (1024*1024):.1f} MB")
    print("\n🚀 KEY FEATURES:")
    print("• Runs offline")
    print("• Optimized for low-resource devices")
    print("• Privacy-focused")

display_model_info()

# Export phrasebook
def export_phrasebook_to_csv(filename="phrasebook.csv"):
    data = []
    for lang, phrases in translator.phrasebook.items():
        for eng, trans in phrases.items():
            data.append({"English": eng, "Language": lang, "Translation": trans})
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"✅ Phrasebook exported to {filename}")
    return df

export_phrasebook_to_csv()

# Phrasebook creation
def create_translation_phrasebook():
    common_phrases = {
        "Greetings": ["Hello", "Good morning", "Good evening"],
        "Basic Needs": ["Where is the bathroom?", "I need help"],
        "Transportation": ["How much does this scooter cost for 1 day?", "Where is the bus station?"],
        "Food & Dining": ["I would like to order", "Can I see the menu?"],
        "Emergency": ["I need a doctor", "Call the police"]
    }

    target_languages = ["Arabic", "Chinese", "Spanish", "French"]
    for lang in target_languages:
        if lang not in translator.phrasebook:
            translator.phrasebook[lang] = {}
        print(f"\n🌍 Creating {lang} phrasebook...")
        for category, phrases in common_phrases.items():
            print(f"📝 Translating {category}...")
            for phrase in phrases:
                if phrase not in translator.phrasebook[lang]:
                    translation = translator.translate(phrase, lang)
                    translator.phrasebook[lang][phrase] = translation

    return translator.phrasebook

print("📚 Creating Multilingual Phrasebook...")
phrasebook = create_translation_phrasebook()

# Translation accuracy test
def test_translation_accuracy():
    test_cases = [
        {
            "english": "Hello, how are you?",
            "arabic_expected": "مرحبا، كيف حالك؟",
            "chinese_expected": "你好，你好吗？"
        },
        {
            "english": "Thank you very much",
            "arabic_expected": "شكرا جزيلا",
            "chinese_expected": "非常感谢"
        },
        {
            "english": "How much does this scooter cost for 1 day?",
            "arabic_expected": "كم تكلفة هذا الدراجة النارية لمدة يوم واحد؟",
            "chinese_expected": "这辆踏板车一天多少钱？"
        }
    ]

    print("🧪 TRANSLATION ACCURACY TESTING")
    print("=" * 50)

    for i, test_case in enumerate(test_cases, 1):
        print(f"\n📝 Test Case {i}:")
        print(f"English: {test_case['english']}")
        arabic_result = translator.translate(test_case['english'], "Arabic")
        chinese_result = translator.translate(test_case['english'], "Chinese")
        print(f"Arabic Result: {arabic_result}")
        print(f"Arabic Expected: {test_case['arabic_expected']}")
        print(f"Chinese Result: {chinese_result}")
        print(f"Chinese Expected: {test_case['chinese_expected']}")
        print("-" * 30)

test_translation_accuracy()

# Troubleshooting guide
def troubleshoot_common_issues():
    print("🔧 TROUBLESHOOTING GUIDE")
    print("=" * 40)
    print("❌ Issue: Model not found")
    print("✅ Solution: Ensure Gemma 3N dataset is added to notebook inputs")
    print("\n❌ Issue: Translation fails")
    print("✅ Solution: Verify TF_LITE_EMBEDDER and TOKENIZER_MODEL are extracted; check phrasebook")
    print("\n❌ Issue: Out of memory")
    print("✅ Solution: Restart kernel and limit input text length")
    print("\n❌ Issue: Language not supported")
    print("✅ Solution: Add translations to phrasebook for the target language")
    print("\n❌ Issue: GPU delegate not working")
    print("✅ Solution: Ensure TensorFlow is up-to-date; verify GPU availability with nvidia-smi")
    print("\n❌ Issue: EMBEDDING_LOOKUP op not supported")
    print("✅ Solution: Update TensorFlow to 2.16.0 or higher; ensure model compatibility")

troubleshoot_common_issues()

# Verify environment and GPU usage
print("\n🔍 Verifying Environment")
print(f"TensorFlow version: {tf.__version__}")
print(f"TensorFlow GPU available: {tf.test.is_gpu_available()}")
print("\n🔍 Verifying GPU Usage")
!nvidia-smi

✅ TensorFlow GPU enabled
✅ Model found at: /kaggle/input/gemma-3n/tflite/gemma-3n-e2b-it-int4/1/gemma-3n-E2B-it-int4.task
📊 Model size: 2990.9 MB
✅ Tokenizer loaded: 262144 tokens
⚠️ GPU delegate not available, using CPU
✅ Embedder loaded
🌍 Supported Languages:
• Arabic (العربية)
• Chinese (中文)
• Spanish (Español)
• French (Français)
• German (Deutsch)
• Italian (Italiano)
• Portuguese (Português)
• Russian (Русский)
• Japanese (日本語)
• Korean (한국어)
• Hindi (हिन्दी)
• Turkish (Türkçe)
• Dutch (Nederlands)
• Swedish (Svenska)
• Norwegian (Norsk)
• Polish (Polski)
• Czech (Čeština)
• Greek (Ελληνικά)
• Hebrew (עברית)
• Thai (ไทย)
🔄 Starting Translation Examples...

🇸🇦 ENGLISH → ARABIC
------------------------------
⏱️ Translation completed in 0.01 seconds (GPU: False)
🇺🇸 EN: How much does this scooter cost for 1 day?
🇸🇦 AR: كم تكلفة هذا الدراجة النارية لمدة يوم واحد؟

⏱️ Translation completed in 0.01 seconds (GPU: False)
🇺🇸 EN: Where is the nearest restaurant?
🇸🇦 AR: أين أقرب مطعم؟

⏱️ Tr

I0000 00:00:1751077411.653676      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [10]:
#!/usr/bin/env python3
"""
Simplified Gemma 3N Translation with TensorFlow Lite
"""

import os
import numpy as np
import tensorflow as tf
import sentencepiece as spm
import zipfile

# Paths
MODEL_PATH = "/kaggle/input/gemma-3n/tflite/gemma-3n-e2b-it-int4/1/gemma-3n-E2B-it-int4.task"
EXTRACT_DIR = "/tmp/gemma3n_extracted"

# Verify model file
if not os.path.exists(MODEL_PATH):
    print(f"❌ Model file not found at {MODEL_PATH}")
    exit(1)
print(f"✅ Model found: {os.path.getsize(MODEL_PATH) / (1024*1024):.1f} MB")

class SimpleTranslator:
    def __init__(self, model_path: str):
        """Initialize translator with TFLite model and tokenizer"""
        self.model_path = model_path
        self.tokenizer = None
        self.interpreter = None
        self._setup()

    def _setup(self):
        """Extract model and load tokenizer and interpreter"""
        try:
            # Extract model components
            os.makedirs(EXTRACT_DIR, exist_ok=True)
            with zipfile.ZipFile(self.model_path, 'r') as zf:
                zf.extractall(EXTRACT_DIR)
            print(f"✅ Model components extracted to {EXTRACT_DIR}")

            # Load tokenizer
            tokenizer_path = os.path.join(EXTRACT_DIR, 'TOKENIZER_MODEL')
            if not os.path.exists(tokenizer_path):
                raise FileNotFoundError(f"Tokenizer model not found at {tokenizer_path}")
            self.tokenizer = spm.SentencePieceProcessor()
            self.tokenizer.Load(tokenizer_path)
            print(f"✅ Tokenizer loaded: {self.tokenizer.GetPieceSize()} tokens")

            # Load TFLite model
            embedder_path = os.path.join(EXTRACT_DIR, 'TF_LITE_EMBEDDER')
            if not os.path.exists(embedder_path):
                raise FileNotFoundError(f"Embedder model not found at {embedder_path}")
            self.interpreter = tf.lite.Interpreter(model_path=embedder_path, num_threads=4)
            self.interpreter.allocate_tensors()
            print("✅ Model loaded (CPU)")

            # Debug: Print input tensor details
            input_details = self.interpreter.get_input_details()
            print(f"🔍 Input tensor details: {input_details}")

        except Exception as e:
            print(f"❌ Setup error: {str(e)}")
            self.interpreter = None
            self.tokenizer = None

    def translate(self, text: str, target_language: str = "Arabic", source_language: str = "English") -> str:
        """Translate text using Gemma 3N model"""
        if not self.interpreter or not self.tokenizer:
            return "❌ Model not initialized"

        # Prepare input prompt for translation
        prompt = f"Translate from {source_language} to {target_language}: {text}"
        tokens = self.tokenizer.EncodeAsIds(prompt)[:128]  # Limit input length
        if not tokens:
            return "❌ Invalid input text"

        # Get input tensor details
        input_details = self.interpreter.get_input_details()
        output_details = self.interpreter.get_output_details()

        # Reshape input to match expected dimensions (e.g., [1, seq_length] or [1, 1])
        # Assume model expects [batch_size, sequence_length]
        input_shape = input_details[0]['shape']
        print(f"🔍 Expected input shape: {input_shape}, Input tokens length: {len(tokens)}")
        
        # Adjust input_data to match expected shape
        if len(input_shape) == 2:  # Expecting [batch_size, sequence_length]
            input_data = np.array([tokens], dtype=np.int32)  # Shape: [1, seq_length]
        else:  # Expecting [batch_size, 1] or similar
            input_data = np.array([[tokens[0]]], dtype=np.int32)  # Single token for iterative processing

        try:
            self.interpreter.set_tensor(input_details[0]['index'], input_data)
            self.interpreter.invoke()
            output_tokens = self.interpreter.get_tensor(output_details[0]['index'])
            translation = self.tokenizer.Decode(output_tokens[0])
            return translation.strip()
        except Exception as e:
            return f"❌ Translation error: {str(e)}"

# Initialize translator
translator = SimpleTranslator(MODEL_PATH)

# Example translations
test_phrases = [
    "Hello, how are you?",
    "How much does this scooter cost for 1 day?",
    "Where is the nearest restaurant?"
]

print("\n🌍 Translation Examples")
print("=" * 40)
for phrase in test_phrases:
    print(f"\n🇺🇸 English: {phrase}")
    arabic = translator.translate(phrase, "Arabic")
    print(f"🇸🇦 Arabic: {arabic}")
    chinese = translator.translate(phrase, "Chinese")
    print(f"🇨🇳 Chinese: {chinese}")

✅ Model found: 2990.9 MB
✅ Model components extracted to /tmp/gemma3n_extracted
✅ Tokenizer loaded: 262144 tokens
✅ Model loaded (CPU)
🔍 Input tensor details: [{'name': 'embedder_token_ids:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1, -1], dtype=int32), 'dtype': <class 'numpy.int32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]

🌍 Translation Examples

🇺🇸 English: Hello, how are you?
🔍 Expected input shape: [1 1], Input tokens length: 12
🇸🇦 Arabic: ❌ Translation error: Cannot set tensor: Dimension mismatch. Got 12 but expected 1 for dimension 1 of input 0.
🔍 Expected input shape: [1 1], Input tokens length: 12
🇨🇳 Chinese: ❌ Translation error: Cannot set tensor: Dimension mismatch. Got 12 but expected 1 for dimension 1 of input 0.

🇺🇸 English: How much does this scooter cost for 1 day?
🔍 Expected input shape: [1 